<a href="https://colab.research.google.com/github/yzhan-met/GPHS-426_RS/blob/main/Radiative_Transfer/RTM_Q3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LibRadtran Tutorial

![logo](http://www.libradtran.org/lib/exe/fetch.php?media=libradtran.png)

libRadtran - A library for radiative transfer - is a collection of C and Fortran functions and programs for calculation of solar and thermal radiation in the Earth's atmosphere. It is a powerful tool used for many cloud/aerosol/land retrieval applications.

[website](https://www.libradtran.org/doku.php)

[paper](https://gmd.copernicus.org/articles/9/1647/2016/)

## Preparing the environment

In [1]:
print("Loading Libraries")
# this cell download and install libRadtran
!apt-get -qq update
!apt-get -qq install -y gfortran flex libnetcdf-dev libgsl-dev gawk

!mkdir ./temp
!curl -SL http://www.libradtran.org/download/libRadtran-2.0.5.tar.gz | tar -xzC ./temp
!mv ./temp/libRadtran-2.0.5 ./libRadtran
!cd ./libRadtran && ./configure && make

from IPython.display import clear_output
clear_output()
print("Setup Complete.")

Setup Complete.


In [2]:
# download additional data for the REPTRAN absorption parameterization
!curl -SL http://www.meteo.physik.uni-muenchen.de/~libradtran/lib/exe/fetch.php?media=download:reptran_2017_all.tar.gz | tar xz -C ./temp
!mv ./temp/data/correlated_k/reptran/* ./libRadtran/data/correlated_k/reptran/ && rm -rf ./temp/data

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   317  100   317    0     0    527      0 --:--:-- --:--:-- --:--:--   528
100  665M  100  665M    0     0  21.4M      0  0:00:31  0:00:31 --:--:-- 23.3M


In [ ]:
!cd libRadtran/examples && ../bin/uvspec < UVSPEC_LOWTRAN_THERMAL.INP > /content/temp/uvspec_lowtran_thermal.out

## Building pyLRT - a python wrapper for libRadtran

In [3]:
print("Building pyLRT...")
!git clone https://github.com/EdGrrr/pyLRT.git && cd pyLRT && pip install .
!echo "/content/libRadtran/">>~/.pylrtrc

clear_output()
print("Setup Complete.")


Setup Complete.


## RT simulation using pyLRT

In [10]:
import copy
import scipy
import scipy.interpolate
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pyLRT import RadTran, get_lrt_folder

pd.options.plotting.backend = "plotly"

LIBRADTRAN_FOLDER = get_lrt_folder()
# set common options
rt = RadTran(LIBRADTRAN_FOLDER)
rt.options['rte_solver'] = 'twostr' # disort

### Question 3 (a)

Report and interpret the differences in the downwelling solar radiation (for wavelengths at 0.64 μm) at the surface for cloud altitudes of 2 km, 5 km, and 10 km.

In [66]:
# initialize solar (shortwave) RT object
sw_rt = copy.deepcopy(rt)
sw_rt.options['source'] = 'solar'
sw_rt.options['wavelength'] = '630 650'
sw_rt.options['output_user'] = 'lambda eglo eup edn edir'
sw_rt.options['albedo'] = '0.2'
sw_rt.options['quiet'] = ''

# Run solar (shortwave) RT for surface simulation
sw_rt.options['zout'] = 'SUR'
sw_rt.options['sza'] = '30'
sdata_SUR, sverb_SUR = sw_rt.run(verbose=True, print_input=True)

# with a cloud at z=10 km
sw_rt_cld10 = copy.deepcopy(sw_rt)
sw_rt_cld10.cloud = {
    "z": np.array([10, 9.7]),
    "lwc": np.array([0.5, 0.5]),
    "re": np.array([20, 20]),
}

sdata_SUR_cld10, sverb_SUR_cld10 = sw_rt_cld10.run(verbose=True, print_input=True)

# with cloud at z=5 km
sw_rt_cld5 = copy.deepcopy(sw_rt)
sw_rt_cld5.cloud = {
    "z": np.array([5, 4.7]),
    "lwc": np.array([0.5, 0.5]),
    "re": np.array([20, 20]),
}

sdata_SUR_cld5, sverb_SUR_cld5 = sw_rt_cld5.run(verbose=True, print_input=True)

# with cloud at z=2 km
sw_rt_cld2 = copy.deepcopy(sw_rt)
sw_rt_cld2.cloud = {
    "z": np.array([2, 1.7]),
    "lwc": np.array([0.5, 0.5]),
    "re": np.array([20, 20]),
}

sdata_SUR_cld2, sverb_SUR_cld2 = sw_rt_cld2.run(verbose=True, print_input=True)

rte_solver twostr
source solar
wavelength 630 650
output_user lambda eglo eup edn edir
albedo 0.2
zout SUR
sza 30
verbose 

rte_solver twostr
source solar
wavelength 630 650
output_user lambda eglo eup edn edir
albedo 0.2
zout SUR
sza 30
wc_file 1D /tmp/tmpbqt35ioo
verbose 
Cloud
  Alt  LWC   Re
 10.00 0.50 20.00
 9.70 0.50 20.00

rte_solver twostr
source solar
wavelength 630 650
output_user lambda eglo eup edn edir
albedo 0.2
zout SUR
sza 30
wc_file 1D /tmp/tmp22cmpx6v
verbose 
Cloud
  Alt  LWC   Re
 5.00 0.50 20.00
 4.70 0.50 20.00

rte_solver twostr
source solar
wavelength 630 650
output_user lambda eglo eup edn edir
albedo 0.2
zout SUR
sza 30
wc_file 1D /tmp/tmp1wk0dhtc
verbose 
Cloud
  Alt  LWC   Re
 2.00 0.50 20.00
 1.70 0.50 20.00



In [67]:
df_clr = pd.DataFrame(sdata_SUR[:, :], columns=['lambda', 'eglo', 'eup', 'edn', 'edir'])
df_cld2 = pd.DataFrame(sdata_SUR_cld2[:, :], columns=['lambda', 'eglo', 'eup', 'edn', 'edir'])
df_cld5 = pd.DataFrame(sdata_SUR_cld5[:, :], columns=['lambda', 'eglo', 'eup', 'edn', 'edir'])
df_cld10 = pd.DataFrame(sdata_SUR_cld10[:, :], columns=['lambda', 'eglo', 'eup', 'edn', 'edir'])


# combine df_TOA["edir"] and df_SUR["eglo"] into a new df
df_plot = pd.DataFrame({'lambda': df_clr["lambda"], 'Clear': df_clr["eglo"], 'Cloud@2km': df_cld2["eglo"], 'Cloud@5km': df_cld5["eglo"], "Cloud@10km": df_cld10["eglo"]})
df_plot.set_index("lambda", inplace=True)

fig = df_plot.plot(y=["Clear", "Cloud@2km", "Cloud@5km", "Cloud@10km"], template="simple_white",
        title="Solar Irradiance", labels=dict(index="", value="Irradiance (W/m2/um)", variable="component"))
fig.update_xaxes(title="Wavelength (nm)")
# set xlim to 250 nm to 5000 nm
# fig.update_layout(xaxis_range=[630, 650], yaxis_range=[500, 800])

### Question 3(b)

In [77]:
# initialize solar (shortwave) RT object
lw_rt = copy.deepcopy(rt)
lw_rt.options['source'] = 'thermal'
lw_rt.options['wavelength'] = '8000 12000'
lw_rt.options['output_user'] = 'lambda edir eup uu'
lw_rt.options['mol_abs_param'] = 'reptran fine'
lw_rt.options['quiet'] = ''

# Run solar (shortwave) RT for surface simulation
lw_rt.options['zout'] = 'TOA'
# lw_rt.options['sza'] = '30'
ldata_TOA, lverb_TOA = lw_rt.run(verbose=True, print_input=True)

# with a cloud at z=10 km
lw_rt_cld10 = copy.deepcopy(lw_rt)
lw_rt_cld10.cloud = {
    "z": np.array([10, 9.7]),
    "lwc": np.array([0.5, 0.5]),
    "re": np.array([20, 20]),
}

ldata_TOA_cld10, lverb_TOA_cld10 = lw_rt_cld10.run(verbose=True, print_input=True)

# with cloud at z=5 km
lw_rt_cld5 = copy.deepcopy(lw_rt)
lw_rt_cld5.cloud = {
    "z": np.array([5, 4.7]),
    "lwc": np.array([0.5, 0.5]),
    "re": np.array([20, 20]),
}

ldata_TOA_cld5, lverb_TOA_cld5 = lw_rt_cld5.run(verbose=True, print_input=True)

# with cloud at z=2 km
lw_rt_cld2 = copy.deepcopy(lw_rt)
lw_rt_cld2.cloud = {
    "z": np.array([2, 1.7]),
    "lwc": np.array([0.5, 0.5]),
    "re": np.array([20, 20]),
}

ldata_TOA_cld2, lverb_TOA_cld2 = lw_rt_cld2.run(verbose=True, print_input=True)

rte_solver twostr
source thermal
wavelength 8000 12000
output_user lambda edir eup uu
mol_abs_param reptran fine
zout TOA
verbose 

rte_solver twostr
source thermal
wavelength 8000 12000
output_user lambda edir eup uu
mol_abs_param reptran fine
zout TOA
wc_file 1D /tmp/tmpa0_j9rue
verbose 
Cloud
  Alt  LWC   Re
 10.00 0.50 20.00
 9.70 0.50 20.00

rte_solver twostr
source thermal
wavelength 8000 12000
output_user lambda edir eup uu
mol_abs_param reptran fine
zout TOA
wc_file 1D /tmp/tmp01a2ptdt
verbose 
Cloud
  Alt  LWC   Re
 5.00 0.50 20.00
 4.70 0.50 20.00

rte_solver twostr
source thermal
wavelength 8000 12000
output_user lambda edir eup uu
mol_abs_param reptran fine
zout TOA
wc_file 1D /tmp/tmp29wfew8j
verbose 
Cloud
  Alt  LWC   Re
 2.00 0.50 20.00
 1.70 0.50 20.00



In [78]:
df_clr = pd.DataFrame(ldata_TOA[:, :], columns=['lambda', 'edir', 'eup'])
df_cld2 = pd.DataFrame(ldata_TOA_cld2[:, :], columns=['lambda', 'edir', 'eup'])
df_cld5 = pd.DataFrame(ldata_TOA_cld5[:, :], columns=['lambda', 'edir', 'eup'])
df_cld10 = pd.DataFrame(ldata_TOA_cld10[:, :], columns=['lambda', 'edir', 'eup'])


# combine into a new df
df_plot = pd.DataFrame({'lambda': df_clr["lambda"], 'Clear': df_clr["eup"], 'Cloud@2km': df_cld2["eup"], 'Cloud@5km': df_cld5["eup"], "Cloud@10km": df_cld10["eup"]})
df_plot.set_index("lambda", inplace=True)

fig = df_plot.plot(y=["Clear", "Cloud@2km", "Cloud@5km", "Cloud@10km"], template="simple_white",
        title="Solar Irradiance", labels=dict(index="", value="Irradiance (W/m2/um)", variable="component"))
fig.update_xaxes(title="Wavelength (nm)")

### Question 3(c)

I will leave this bonus question for you to have a try, using Q3(a) and Q3(b) as references. Also check http://libradtran.org/doc/libRadtran.pdf for more information.